In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Load data
train_df = pd.read_csv("../sales_train.csv.gz")
test_df  = pd.read_csv("../test.csv.gz")

categories = pd.read_csv('../item_categories.csv')
items = pd.read_csv('../items.csv')
shops = pd.read_csv('../shops.csv')

In [2]:
%store -r __prepare_data_4

In [3]:
%%time
__prepare_data_4

CPU times: user 3min, sys: 1min 55s, total: 4min 56s
Wall time: 5min 13s


In [4]:
# based on tables from 2019.11.16/trends_combined.ipynb I choose those shops to be taken out:
closed_shops = [0, 1, 8, 9, 11, 13, 17, 20, 23,  27,  29, 30, 32, 33,  36,  40, 43, 54]

# one of those, though, needs to be predicted (#36):
print(test_df[test_df.shop_id.isin(closed_shops)].shop_id.unique())

# and one of those (#51) does not (keep it for training anyways):
print(set(all_data.shop_id.unique()).difference(test_df.shop_id.unique()))

# this explains why shop #36 is not a good shop:
(all_data[all_data.shop_id == 36]
             .groupby(["date_block_num"])
             .agg({'total_monthly_shop_sales': lambda x: np.NA if x.empty else x.iloc[0]})
             .reset_index())

[36]
{0, 1, 8, 9, 11, 13, 17, 20, 23, 27, 29, 30, 32, 33, 40, 43, 51, 54}


,date_block_num,total_monthly_shop_sales
0,33,0.060964
1,34,0.000000


In [5]:
from calendar import monthrange

# January, 2013 is date_block_num==0
timeline = pd.DataFrame({'begin_date': pd.date_range(start='2013-01-01',end='2015-11-01', freq='MS'),
                         'date_block_num': range(35)} )
    
timeline['n_days'] = timeline.begin_date.apply(lambda date: monthrange(date.year,date.month)[1])

timeline['workdays'] = [17, 20, 20, 22, 18, 19, 23, 22, 21, 23, 20, 22,
                        17, 20, 20, 22, 19, 19, 23, 21, 22, 23, 18, 23,
                        15, 19, 21, 22, 18, 21, 23, 21, 22, 22, 20]

timeline['month_num'] = timeline['date_block_num'] % 12

In [22]:
%%time
# the idea is to aggregate rarely sold items over larger periods (e.g. quarters) also identifying discontinued items

agg["target"]  = agg["target"].astype(np.float16)
agg['shop_id'] = agg['shop_id'].astype(np.int8)
agg['item_id'] = agg['item_id'].astype(np.int16)

sales = pd.merge(agg, timeline[:-1], how='right', on=['date_block_num']).sort_values(by=['date_block_num','shop_id','item_id'])

def check_series(x):
    start = min( np.where(~ np.isnan(x))[0] )
    end   = max( np.where(~ np.isnan(x))[0] )

    no_stock = 1
    discontinued = 2
    retval = 0

    # last months sales are present, but not enough lags for seasonal inference => ignore
    if start+12 >= end:
        retval |= no_stock

    # there should be sales in last month, otherwise consider it is discontinued
    if end < 33:
        retval |= discontinued
        
    # at least last 3 month with sales should be present
    if sum(~ np.isnan(x[-3:])) < 3:
        retval |= no_stock

    return retval

series = sales.groupby(['shop_id','item_id']).agg({'target': check_series})

CPU times: user 6min, sys: 2.09 s, total: 6min 2s
Wall time: 6min 4s


In [30]:
len(np.where(series.target<1)[0])

102